In [1]:
import pandas as pd 
import numpy as np
import glob
import os
import re
import matplotlib.pyplot as plt 
from datetime import datetime, timedelta
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics
from sklearn.metrics import f1_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
#from pydrive.auth import GoogleAuth
#from pydrive.drive import GoogleDrive

Informal ReadMe: 
- My top priority is not scalabilty (yet) 
- 

In [2]:
# Michael's Advice: 
    # One df per patient
    # Pivot? Get dummies?
    # Rolling Window   !!! Ben said 3 second window is best
    
# JM's Advice: 
    # Write a function to bring in all the data
    # Have it make a patient column and doc column
    # But getting an MVP could be super useful too. 
        # So if you want to try the experiment on pt 1, that's good too. 

# Experimentation with patient #1

CPG #1

In [3]:
CPG1 = pd.read_csv('Barretts_Analysis/CPG/001 CPG.csv', skiprows = 7).fillna(' ')
# Maybe don't need fillna()

# Create a new variable called 'header' from the first row of the dataset
header = CPG1.iloc[0]
header = header.str.replace('\n', ' ')
CPG1 = CPG1.rename(columns = header)
CPG1 = CPG1[1:]

CPG1 = CPG1.drop(columns = ['Position from LES (cm)', 'BCT (sec)', 'pH Delta (pH)', 'ACT (sec)'])

CPG1 = CPG1[CPG1["Meas Type"].str.contains("MII")]
CPG1 = CPG1.rename(columns={'Meas Start Time': 'CPG Start Time'})

In [4]:
Auto1 = pd.read_csv('Barretts_Analysis/Autoscan/001 Autoscan.csv', skiprows = 7).fillna(' ')
# Maybe don't need fillna()

# Create a new variable called 'header' from the first row of the dataset
header = Auto1.iloc[0]
header = header.str.replace('\n', ' ')
Auto1 = Auto1.rename(columns = header)
Auto1 = Auto1[1:]

Auto1 = Auto1.drop(columns = ['Position from LES (cm)', 'BCT (sec)', 'pH Delta (pH)', 'ACT (sec)'])

Auto1 = Auto1[Auto1["Meas Type"].str.contains("MII")]
Auto1 = Auto1.rename(columns={'Meas Start Time': 'Auto Start Time'})

In [5]:
Auto1.shape

(39, 8)

# Test 1: merge

# Test 2: Concatenating CPG + Auto - Patient 1

In [6]:
TestCPGAuto2 = pd.concat([CPG1, Auto1], axis = 1).fillna('').drop(columns = {'Meas Type', 'Body Position', 'Contents'})

In [7]:
TestCPGAuto2.shape

(63, 10)

In [8]:
# If you have a Doc column again and stack vertically, you could sort by start time. 
# Probably need Start Day and Start Time re.split

In [9]:
TestCPGAuto2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 63 entries, 1 to 377
Data columns (total 10 columns):
Meas                 63 non-null object
CPG Start Time       63 non-null object
MII Reflux           63 non-null object
pH Classification    63 non-null object
                     63 non-null object
Meas                 63 non-null object
Auto Start Time      63 non-null object
MII Reflux           63 non-null object
pH Classification    63 non-null object
                     63 non-null object
dtypes: object(10)
memory usage: 5.4+ KB


# Test 3: Concatenate vertically - CPG, Auto - Patient 1

In [10]:
CPG1b = pd.read_csv('Barretts_Analysis/CPG/001 CPG.csv', skiprows = 7).fillna(' ')
# Maybe don't need fillna()

# Create a new variable called 'header' from the first row of the dataset
header = CPG1b.iloc[0]
header = header.str.replace('\n', ' ')
CPG1b = CPG1b.rename(columns = header)
CPG1b = CPG1b[1:]

CPG1b = CPG1b.drop(columns = ['Position from LES (cm)', 'BCT (sec)', 'pH Delta (pH)', 'ACT (sec)'])

CPG1b = CPG1b[CPG1b["Meas Type"].str.contains("MII")]
CPG1b['Doc'] = 'CPG'

Auto1b = pd.read_csv('Barretts_Analysis/Autoscan/001 Autoscan.csv', skiprows = 7).fillna(' ')
# Maybe don't need fillna()

# Create a new variable called 'header' from the first row of the dataset
header = Auto1b.iloc[0]
header = header.str.replace('\n', ' ')
Auto1b = Auto1b.rename(columns = header)
Auto1b = Auto1b[1:]

Auto1b = Auto1b.drop(columns = ['Position from LES (cm)', 'BCT (sec)', 'pH Delta (pH)', 'ACT (sec)'])

Auto1b = Auto1b[Auto1b["Meas Type"].str.contains("MII")]
Auto1b['Doc'] = 'Auto'

In [11]:
TestCPGAuto3 = pd.concat([CPG1b, Auto1b]).fillna('').drop(columns = {'Meas Type', 'Body Position', 'Contents'})

In [12]:
TestCPGAuto3

,Meas,Meas Start Time,MII Reflux,pH Classification,,Doc
1,Meas 1:,1/16:42:24.0,Reflux,Acid,,CPG
12,Meas 4:,1/16:46:55.6,Reflux,Acid,,CPG
21,Meas 6:,1/16:57:22.0,Reflux,Acid,,CPG
30,Meas 8:,1/18:08:27.0,Reflux,NonAcid,,CPG
36,Meas 9:,1/18:32:44.0,Reflux,Acid,,CPG
...,...,...,...,...,...,...
329,Meas 80:,2/11:57:13.0,Reflux,Acid,,Auto
341,Meas 83:,2/12:08:02.6,No Reflux,Needs pH,,Auto
349,Meas 86:,2/12:18:02.0,Reflux,Acid,,Auto
365,Meas 91:,2/12:32:51.0,Reflux,Acid,,Auto


In [13]:
TestCPGAuto3.Doc.value_counts()

Auto    39
CPG     37
Name: Doc, dtype: int64

Remember: 
    # If you have a Doc column again and stack vertically, you could sort by start time. 
    # Probably need Start Day and Start Time re.split

* Could I write a function to read in the files and label them with the patient number? And Doc name? 

* Do I still need to split the start time? Should it be a datetime object? 

* Could you do ML to find out who is predicting better? Reflux = 1, No Reflux = 0

# Try to bring in all data & label accordingly 

In [14]:
import pandas as pd
import glob

path = 'Barretts_Analysis/'
next_path = glob.glob(path + "*")
#all_files = glob.glob(next_path + "/*.csv")



# for filename in all_files:
#     df = pd.read_csv(filename, index_col=None, header=0)
#     li.append(df)

# frame = pd.concat(li, axis=0, ignore_index=True)
# command + ? = BIG comment

li = []

for some_path in next_path: 
    #print(some_path) 
    if some_path == 'Barretts_Analysis/AI': 
        continue
    else: 
        for file in glob.glob(some_path + "/*.csv"):
            #print(file)
            df = pd.read_csv(file, index_col=None, header=0, skiprows = 7).fillna('-')
            header = df.iloc[0]
            header = header.str.replace('\n', ' ')
            df = df.rename(columns = header)
            df = df[1:]
            df = df.drop(columns = ['Position from LES (cm)', 'BCT (sec)', 'pH Delta (pH)', 'ACT (sec)'])
            df = df[df["Meas Type"].str.contains("MII")]
            #df['patient_number'] = file.replace('/', ' ')
            df['z_filename'] = file.replace(' ', '-')
            #df['z_filename'] = df['z_filename'].dropna()
            #print(df.head())
            li.append(df)
        
frame = pd.concat(li, axis=0, ignore_index=True)
# command + ? = BIG comment

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:39: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.



But I don't have my doc name or patient number....!

In [15]:
frame.z_filename.value_counts()

Barretts_Analysis/Autoscan/019-Autoscan.csv    197
Barretts_Analysis/ES/019-ES.csv                189
Barretts_Analysis/SR/019-SR.csv                171
Barretts_Analysis/MF/019---MF.csv              157
Barretts_Analysis/ES/015-ES.csv                153
                                              ... 
Barretts_Analysis/DS/001-DS.csv                 37
Barretts_Analysis/SR/001-SR.csv                 37
Barretts_Analysis/CPG/001-CPG.csv               37
Barretts_Analysis/MF/001---MF.csv               31
Barretts_Analysis/CPG/007-CPG.csv               25
Name: z_filename, Length: 115, dtype: int64

In [16]:
frame.z_filename[0]

'Barretts_Analysis/Autoscan/019-Autoscan.csv'

In [17]:
frame[['patient_num','doc', 'None', 'None']] = frame.z_filename.str.split("-",expand=True) 

In [18]:
frame[['folder1', 'doctor', 'patient']] = frame.patient_num.str.split('/', expand = True).replace('')

In [19]:
frame['patient2'] = frame.patient_num.str.extract('(\d+)').astype(int)

In [20]:
frame = frame.drop(columns = {'Meas Type', 'Body Position', 'Contents', 'patient_num', 'doc', 'None', 'folder1', 'z_filename', 'patient', '-'})

In [21]:
frame

,MII Reflux,Meas,Meas Start Time,pH Classification,doctor,patient2
0,Reflux,Meas 1:,1/6:58:52.0,NonAcid,Autoscan,19
1,Reflux,Meas 2:,1/7:17:45.0,Acid,Autoscan,19
2,Reflux,Meas 3:,1/7:44:58.0,NonAcid,Autoscan,19
3,Reflux,Meas 4:,1/7:46:44.0,NonAcid,Autoscan,19
4,Reflux,Meas 5:,1/7:48:10.0,NonAcid,Autoscan,19
...,...,...,...,...,...,...
9086,Reflux,Meas 116:,2/9:12:11.0,Acid,ES,13
9087,Reflux,Meas 118:,2/9:22:21.0,Acid,ES,13
9088,Reflux,Meas 120:,2/9:26:41.0,Acid,ES,13
9089,Reflux,Meas 122:,2/10:01:51.8,Acid,ES,13


In [22]:
# Split meas start time and convert to datetime. Convert to 24 hour? 

In [23]:
frame.patient2.value_counts()

19    1012
15     761
11     684
20     669
3      633
12     504
4      494
18     488
9      447
5      420
8      395
10     394
17     394
13     339
16     321
6      320
14     302
2      264
1      225
7       25
Name: patient2, dtype: int64

In [24]:
frame.doctor.value_counts()

Autoscan    1719
ES          1675
SR          1536
CPG         1453
DS          1376
MF          1332
Name: doctor, dtype: int64

In [25]:
frame[['Meas Start Day', 'Meas Start Time']] = frame['Meas Start Time'].str.split("/",expand=True)

In [26]:
frame.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9091 entries, 0 to 9090
Data columns (total 7 columns):
MII Reflux           9091 non-null object
Meas                 9091 non-null object
Meas Start Time      9091 non-null object
pH Classification    9091 non-null object
doctor               9091 non-null object
patient2             9091 non-null int64
Meas Start Day       9091 non-null object
dtypes: int64(1), object(6)
memory usage: 497.3+ KB


In [27]:
# maybe 

# frame['Meas Start Time 2'] = pd.to_datetime(frame['Meas Start Time'], format = '%H:%M:%S.%f')
#frame['Meas Start Time 2'] = pd.to_datetime(frame['Meas Start Time'])

In [28]:
frame

,MII Reflux,Meas,Meas Start Time,pH Classification,doctor,patient2,Meas Start Day
0,Reflux,Meas 1:,6:58:52.0,NonAcid,Autoscan,19,1
1,Reflux,Meas 2:,7:17:45.0,Acid,Autoscan,19,1
2,Reflux,Meas 3:,7:44:58.0,NonAcid,Autoscan,19,1
3,Reflux,Meas 4:,7:46:44.0,NonAcid,Autoscan,19,1
4,Reflux,Meas 5:,7:48:10.0,NonAcid,Autoscan,19,1
...,...,...,...,...,...,...,...
9086,Reflux,Meas 116:,9:12:11.0,Acid,ES,13,2
9087,Reflux,Meas 118:,9:22:21.0,Acid,ES,13,2
9088,Reflux,Meas 120:,9:26:41.0,Acid,ES,13,2
9089,Reflux,Meas 122:,10:01:51.8,Acid,ES,13,2


In [29]:
frame.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9091 entries, 0 to 9090
Data columns (total 7 columns):
MII Reflux           9091 non-null object
Meas                 9091 non-null object
Meas Start Time      9091 non-null object
pH Classification    9091 non-null object
doctor               9091 non-null object
patient2             9091 non-null int64
Meas Start Day       9091 non-null object
dtypes: int64(1), object(6)
memory usage: 497.3+ KB


In [30]:
#frame['Meas Start Time 2'].str.strip('T00:00:00')

In [31]:
frame

,MII Reflux,Meas,Meas Start Time,pH Classification,doctor,patient2,Meas Start Day
0,Reflux,Meas 1:,6:58:52.0,NonAcid,Autoscan,19,1
1,Reflux,Meas 2:,7:17:45.0,Acid,Autoscan,19,1
2,Reflux,Meas 3:,7:44:58.0,NonAcid,Autoscan,19,1
3,Reflux,Meas 4:,7:46:44.0,NonAcid,Autoscan,19,1
4,Reflux,Meas 5:,7:48:10.0,NonAcid,Autoscan,19,1
...,...,...,...,...,...,...,...
9086,Reflux,Meas 116:,9:12:11.0,Acid,ES,13,2
9087,Reflux,Meas 118:,9:22:21.0,Acid,ES,13,2
9088,Reflux,Meas 120:,9:26:41.0,Acid,ES,13,2
9089,Reflux,Meas 122:,10:01:51.8,Acid,ES,13,2


# Dibran's code for shifting

In [39]:
frame['Meas Start TimeStamp'] = pd.to_datetime(frame['Meas Start Time'])
frame['3_second_shift'] = frame['Meas Start TimeStamp'] + timedelta(seconds = -3)
frame['3_second_shift_forward'] = frame['Meas Start TimeStamp'] + timedelta(seconds = 3)
frame['3_second_target'] = 0
new_frame = frame.groupby('doctor')

In [40]:
%%time
filtered_rows = pd.DataFrame([])
for k in new_frame.groups.keys():
    data = new_frame.get_group(k)
    for x, y in data.iterrows():
        if y['MII Reflux'] =='Reflux':
            the_only_hope = data[(data['Meas Start TimeStamp'] > y['3_second_shift'])&
                              (data['Meas Start TimeStamp'] < y['Meas Start TimeStamp'])]
        else: 
            continue
    filtered_rows = filtered_rows.append(the_only_hope)

CPU times: user 20.7 s, sys: 405 ms, total: 21.1 s
Wall time: 24.6 s


In [49]:
frame.head()

,MII Reflux,Meas,Meas Start Time,pH Classification,doctor,patient2,Meas Start Day,Meas Start TimeStamp,3_second_shift,3_second_target,3_second_shift_forward
0,Reflux,Meas 1:,6:58:52.0,NonAcid,Autoscan,19,1,2020-05-14 06:58:52,2020-05-14 06:58:49,0,2020-05-14 06:58:55
1,Reflux,Meas 2:,7:17:45.0,Acid,Autoscan,19,1,2020-05-14 07:17:45,2020-05-14 07:17:42,0,2020-05-14 07:17:48
2,Reflux,Meas 3:,7:44:58.0,NonAcid,Autoscan,19,1,2020-05-14 07:44:58,2020-05-14 07:44:55,0,2020-05-14 07:45:01
3,Reflux,Meas 4:,7:46:44.0,NonAcid,Autoscan,19,1,2020-05-14 07:46:44,2020-05-14 07:46:41,0,2020-05-14 07:46:47
4,Reflux,Meas 5:,7:48:10.0,NonAcid,Autoscan,19,1,2020-05-14 07:48:10,2020-05-14 07:48:07,0,2020-05-14 07:48:13


In [72]:
frame['Meas Start Day'] = frame['Meas Start Day'].astype('int64')

In [81]:
frame = frame.sort_values(["patient2", "Meas Start Day", "Meas Start Time"], ascending = (True, True, True))
frame.head(50)
# consider using 4 seconds instead of 3

,MII Reflux,Meas,Meas Start Time,pH Classification,doctor,patient2,Meas Start Day,Meas Start TimeStamp,3_second_shift,3_second_target,3_second_shift_forward
8514,Reflux,Meas 1:,16:12:40.4,MinorAcid,ES,1,1,2020-05-14 16:12:40.400,2020-05-14 16:12:37.400,0,2020-05-14 16:12:43.400
5941,Reflux,Meas 5:,16:12:40.5,NonAcid,SR,1,1,2020-05-14 16:12:40.500,2020-05-14 16:12:37.500,0,2020-05-14 16:12:43.500
5586,Reflux,Meas 1:,16:35:31.0,NonAcid,DS,1,1,2020-05-14 16:35:31.000,2020-05-14 16:35:28.000,0,2020-05-14 16:35:34.000
705,Reflux,Meas 1:,16:35:31.0,NonAcid,Autoscan,1,1,2020-05-14 16:35:31.000,2020-05-14 16:35:28.000,0,2020-05-14 16:35:34.000
5587,Reflux,Meas 2:,16:42:24.0,Acid,DS,1,1,2020-05-14 16:42:24.000,2020-05-14 16:42:21.000,0,2020-05-14 16:42:27.000
8515,Reflux,Meas 2:,16:42:24.0,Acid,ES,1,1,2020-05-14 16:42:24.000,2020-05-14 16:42:21.000,0,2020-05-14 16:42:27.000
1828,Reflux,Meas 9:,16:42:24.0,Acid,MF,1,1,2020-05-14 16:42:24.000,2020-05-14 16:42:21.000,0,2020-05-14 16:42:27.000
3912,Reflux,Meas 1:,16:42:24.0,Acid,CPG,1,1,2020-05-14 16:42:24.000,2020-05-14 16:42:21.000,0,2020-05-14 16:42:27.000
706,Reflux,Meas 2:,16:42:24.0,Acid,Autoscan,1,1,2020-05-14 16:42:24.000,2020-05-14 16:42:21.000,0,2020-05-14 16:42:27.000
5942,Reflux,Meas 10:,16:42:24.0,Acid,SR,1,1,2020-05-14 16:42:24.000,2020-05-14 16:42:21.000,0,2020-05-14 16:42:27.000


In [83]:
frame = frame.reset_index(drop=True)

## sort by (patient?) then by meas time , write a func starting on first row and iterate through rows, for each row, see if next row occurs within three sec. If yes, label meas. if not, increase by one

## dont forget same day, same patient 

if current_item_time is +- 3 sec of previous_item_time: 
    and Meas Start Day == previous Meas Start Day:
        label Measurement (n)
    
    

In [ ]:
for item, frame in df['Meas Start TimeStamp'].iteritems():
    if next <= '00:00:03.000':
        print frame
        2020-05-14 16:12:40.400

In [ ]:
[(df[1].ge(df[1].shift())) & df[1].le(df[1].shift(-1))]

In [87]:
def read_row(df):

    if df['Meas Start Time'].current == df['Meas Start Time'].next:
        df['Meas No.']' = 1
    elif df['Meas Start Time'].next <= 3s of df['Meas Start Time'].current:
        df['Meas No.'] = '1'
    else df['Meas No.'].value = df['Meas No.'].next



SyntaxError: EOL while scanning string literal (<ipython-input-87-e3c2fe297b0b>, line 4)

In [88]:
def read_row_2(df):
    for row in df.iterrows:
        if (row['Meas Start TimeStamp'].next - row['Meas Start TimeStamp'].current).dt.total_seconds() <= 3.00:
            row['Meas No.'] = 1
        else: 
            row['Meas No.'] = row['Meas No.'] + 1          

Nested loop, use groupbys instead of for loops (grupby patient and day)
Pandas.cut to do a groupby arange on time
for each group, iterate over the groups and add labels 

In [91]:
by_pt_dy = frame.groupby(['patient2', 'Meas Start Day'])

for patient, startday in by_pt_dy: 
    print(f"First 2 entries for {patient!r}")
    print("------------------------")
    print(startday.head(2), end="\n\n")

First 2 entries for (1, 1)
------------------------
  MII Reflux     Meas Meas Start Time pH Classification doctor  patient2  \
0     Reflux  Meas 1:      16:12:40.4         MinorAcid     ES         1   
1     Reflux  Meas 5:      16:12:40.5           NonAcid     SR         1   

   Meas Start Day    Meas Start TimeStamp          3_second_shift  \
0               1 2020-05-14 16:12:40.400 2020-05-14 16:12:37.400   
1               1 2020-05-14 16:12:40.500 2020-05-14 16:12:37.500   

   3_second_target  3_second_shift_forward  
0                0 2020-05-14 16:12:43.400  
1                0 2020-05-14 16:12:43.500  

First 2 entries for (1, 2)
------------------------
    MII Reflux      Meas Meas Start Time pH Classification doctor  patient2  \
149     Reflux  Meas 75:      10:38:30.0              Acid     MF         1   
150     Reflux  Meas 71:      10:38:30.0              Acid    CPG         1   

     Meas Start Day Meas Start TimeStamp      3_second_shift  3_second_target  \
149 

     MII Reflux       Meas Meas Start Time pH Classification doctor  patient2  \
5051     Reflux  Meas 104:      10:01:43.0              Acid     MF        13   
5052     Reflux  Meas 125:      10:01:43.0              Acid     SR        13   

      Meas Start Day Meas Start TimeStamp      3_second_shift  \
5051               2  2020-05-14 10:01:43 2020-05-14 10:01:40   
5052               2  2020-05-14 10:01:43 2020-05-14 10:01:40   

      3_second_target 3_second_shift_forward  
5051                0    2020-05-14 10:01:46  
5052                0    2020-05-14 10:01:46  

First 2 entries for (14, 1)
------------------------
     MII Reflux     Meas Meas Start Time pH Classification doctor  patient2  \
5144     Reflux  Meas 1:      15:25:53.1           NonAcid     DS        14   
5145     Reflux  Meas 4:      16:13:06.0              Acid     SR        14   

      Meas Start Day    Meas Start TimeStamp          3_second_shift  \
5144               1 2020-05-14 15:25:53.100 2020-05-14

In [93]:
frame[frame['Meas Start Day'] == 2]

,MII Reflux,Meas,Meas Start Time,pH Classification,doctor,patient2,Meas Start Day,Meas Start TimeStamp,3_second_shift,3_second_target,3_second_shift_forward
149,Reflux,Meas 75:,10:38:30.0,Acid,MF,1,2,2020-05-14 10:38:30,2020-05-14 10:38:27,0,2020-05-14 10:38:33
150,Reflux,Meas 71:,10:38:30.0,Acid,CPG,1,2,2020-05-14 10:38:30,2020-05-14 10:38:27,0,2020-05-14 10:38:33
151,Reflux,Meas 74:,10:38:30.0,Acid,Autoscan,1,2,2020-05-14 10:38:30,2020-05-14 10:38:27,0,2020-05-14 10:38:33
152,Reflux,Meas 78:,10:38:30.0,Acid,ES,1,2,2020-05-14 10:38:30,2020-05-14 10:38:27,0,2020-05-14 10:38:33
153,Reflux,Meas 87:,10:38:30.0,Acid,SR,1,2,2020-05-14 10:38:30,2020-05-14 10:38:27,0,2020-05-14 10:38:33
...,...,...,...,...,...,...,...,...,...,...,...
9086,Reflux,Meas 268:,5:26:31.0,NonAcid,Autoscan,20,2,2020-05-14 05:26:31,2020-05-14 05:26:28,0,2020-05-14 05:26:34
9087,Reflux,Meas 269:,5:34:19.0,NonAcid,Autoscan,20,2,2020-05-14 05:34:19,2020-05-14 05:34:16,0,2020-05-14 05:34:22
9088,Reflux,Meas 270:,5:35:14.0,NonAcid,Autoscan,20,2,2020-05-14 05:35:14,2020-05-14 05:35:11,0,2020-05-14 05:35:17
9089,Reflux,Meas 271:,5:52:08.0,NonAcid,Autoscan,20,2,2020-05-14 05:52:08,2020-05-14 05:52:05,0,2020-05-14 05:52:11
